<a href="https://colab.research.google.com/github/OLatinwo/REU-DataScienceProgram/blob/main/DataWrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lecture 1: Wrangling
  In this lecture notebook, we will test the hypothesis that the CEOs of major companies are typically in their 40s or older. To do this, we will identify top companies, find their CEOS, extract their ages, and look at the distribution of their ages. 

In [ ]:
#Lets start by installing some libraries that are useful for programming data

#For crawling pages
!pip install scrapy

#Optional, for parallel excecution
!pip install swifter

#For string similarity
!pip install py_stringsimjoin

# lxml to parse xml tree
!pip install lxml

!pip install pandas
!pip install numpy
!pip install matplotlib

     |████████████████████████████████| 256kB 4.9MB/s 
     |████████████████████████████████| 3.2MB 20.0MB/s 
     |████████████████████████████████| 3.2MB 37.4MB/s 
     |████████████████████████████████| 256kB 35.1MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
     |████████████████████████████████| 3.1MB 34.6MB/s 
     |████████████████████████████████| 71kB 713kB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
  Created wheel for protego: filename=Protego-0.1.16-cp37-none-any.whl size=7783 sha256=304b1daf001bbeefc2f0339ba334227360d227fb1d9d88991d840e9817b67ae5
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp37-none-any.whl size=11517 sha256=c6bb416b0f0e79b2263109b7cae1f96ed0142f51f0a5ec281dec21db6242f1e5
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
Successfully built protego Py

     |████████████████████████████████| 1.1MB 6.5MB/s 
     |████████████████████████████████| 665kB 19.5MB/s 
  Created wheel for py-stringsimjoin: filename=py_stringsimjoin-0.3.2-cp37-cp37m-linux_x86_64.whl size=3723790 sha256=89d192e30c50c6d4aeecbef475f3169e0d14f7c2f4c5066362a5c8a549a53380
  Stored in directory: /root/.cache/pip/wheels/58/30/8e/6fa463100ff6e8595b131067ec43a118c607f621ac297e7dad
  Created wheel for py-stringmatching: filename=py_stringmatching-0.4.2-cp37-cp37m-linux_x86_64.whl size=2055261 sha256=47e38cd44496811829817225ba0aaa81cad66b1bb6090e07068e29ab863de351
  Stored in directory: /root/.cache/pip/wheels/be/8c/5d/bcdaf17d0784738b73e856ebc4061cd3f2509753a83dd71058
Successfully built py-stringsimjoin py-stringmatching


In [ ]:
# Here are some imports for simplicity
# For parsing dates and being able to compare
import datetime

# For fetching remote data
import urllib

#Pandas dataframes
import pandas as pd

#Numpy matrix and array operations
import numpy as np

#Sqlite is a simplistic database
import sqlite3

#Crawler for multiple web pages at once
import scrapy
from scrapy.crawler import CrawlerProcess

#Try to parallalize the comptation
import swifter

#Approximate String Matching
import py_stringsimjoin as ssj
import py_stringmatching as sm

#Data Visualization
import matplotlib

#import urllib and etree for download and parsing 
from lxml import etree

# Acquiring Data About Companies and CEOs

In [ ]:
data = urllib.request.urlopen(\
      'https://gist.githubusercontent.com/jvilledieu/c3afe5bc21da28880a30/raw/a344034b82a11433ba6f149afa47e57567d4a18f/Companies.csv')

company_data_df = pd.read_csv(data)

In [ ]:
#Persist it so an SQLite database, and read it back.  
conn = sqlite3.connect('local.db')

company_data_df.to_sql("companies", conn, if_exists="replace", index=False)

pd.read_sql_query('select * from companies', conn)

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at
0,/organization/waywire,#waywire,http://www.waywire.com,|Entertainment|Politics|Social Media|News|,News,1 750 000,acquired,USA,NY,New York City,New York,1,01/06/2012,2012-06,2012-Q2,2012.0,30/06/2012,30/06/2012
1,/organization/tv-communications,&TV Communications,http://enjoyandtv.com,|Games|,Games,4 000 000,operating,USA,CA,Los Angeles,Los Angeles,2,None,None,None,NaN,04/06/2010,23/09/2010
2,/organization/rock-your-paper,'Rock' Your Paper,http://www.rockyourpaper.org,|Publishing|Education|,Publishing,40 000,operating,EST,None,Tallinn,Tallinn,1,26/10/2012,2012-10,2012-Q4,2012.0,09/08/2012,09/08/2012
3,/organization/in-touch-network,(In)Touch Network,http://www.InTouchNetwork.com,|Electronics|Guides|Coffee|Restaurants|Music|iPhone|Apps|Mobile|iOS|E-Commerce|,Electronics,1 500 000,operating,GBR,None,London,London,1,01/04/2011,2011-04,2011-Q2,2011.0,01/04/2011,01/04/2011
4,/organization/n-plusn,+n (PlusN),http://plusn.com,|Software|,Software,1 200 000,operating,USA,NY,New York City,New York,2,01/01/2012,2012-01,2012-Q1,2012.0,29/08/2012,04/09/2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47753,/organization/zzish,Zzish,http://www.zzish.com,|Analytics|Gamification|Developer APIs|iOS|Android|Education|,Education,320 000,operating,GBR,None,London,London,1,28/01/2013,2013-01,2013-Q1,2013.0,24/03/2014,24/03/2014
47754,/organization/zznode-science-and-technology-co-ltd,ZZNode Science and Technology,http://www.zznode.com,|Enterprise Software|,Enterprise Software,1 587 301,operating,CHN,None,Beijing,Beijing,1,None,None,None,NaN,01/04/2012,01/04/2012
47755,/organization/zzzzapp-com,Zzzzapp Wireless ltd.,http://www.zzzzapp.com,|Web Development|Advertising|Wireless|Mobile|,Web Development,97 398,operating,HRV,None,Split,Split,5,13/05/2012,2012-05,2012-Q2,2012.0,01/11/2011,10/09/2014
47756,/organization/a-list-games,[a]list games,http://www.alistgames.com,|Games|,Games,9 300 000,operating,None,None,None,None,1,None,None,None,NaN,21/11/2011,21/11/2011


In [ ]:
# Now lets read a HTML table containing information about CEOs
company_ceos_df = pd.read_html('https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autoload-data/data-wrangling/en.wikipedia.org/wiki/List_of_chief_executive_officers.html#List_of_CEOs')[0]
company_ceos_df

,Company,Executive,Title,Since,Notes,Updated
0,Accenture,Julie Sweet,Interim CEO[1],2019,"Succeeded Pierre Nanterme, Passed Away",2019-01-31
1,Aditya Birla Group,Kumar Birla,Chairman[2],1995[2],Part of the Birla family business house in India,2018-10-01
2,Adobe Systems,Shantanu Narayen,"Chairman, president and CEO[3]",2007,Formerly with Apple Inc.,2018-10-01
3,Agenus,Garo H. Armen,"Founder, chairman, CEO[4]",1994,Founder of the Children of Armenia Fund (COAF),2018-10-01
4,Airbus,Tom Enders,CEO[5],2012,Succeeded Louis Gallois,2017-11-14
...,...,...,...,...,...,...
180,Williams-Sonoma,Laura J. Alber,President and CEO[168],2010,Replaced W. Howard Lester,2017-11-11
181,Wipro Technologies Limited,Abidali Neemuchwala,CEO[169],2016,Previously with Tata Consultancy Services,2017-11-11
182,WWE,Vince McMahon,Chairman and CEO[170],2009,Chairman of the executive committeeChairman since 1980President and CEO from 1980-1993,2017-11-11
183,Yum! Brands,Greg Creed,CEO[171],2015,Previously CEO for Taco Bell,2017-11-11


In [ ]:
crawl_list = []
for executive in company_ceos_df['Executive']:
  crawl_list.append('https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autoload-data/data-wrangling/en.wikipedia.org/wiki/' + executive.replace(' ', '_') + '.html')
crawl_list

['https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autoload-data/data-wrangling/en.wikipedia.org/wiki/Julie_Sweet.html',
 'https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autoload-data/data-wrangling/en.wikipedia.org/wiki/Kumar_Birla.html',
 'https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autoload-data/data-wrangling/en.wikipedia.org/wiki/Shantanu_Narayen.html',
 'https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autoload-data/data-wrangling/en.wikipedia.org/wiki/Garo_H._Armen.html',
 'https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autoload-data/data-wrangling/en.wikipedia.org/wiki/Tom_Enders.html',
 'https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autoload-data/data-wrangling/en.wikipedia.org/wiki/Daniel_Zhang.html',
 'https://raw.githubu

In [ ]:
# use urllib to crawl all pages in crawl_list, and store the response of page
# in list_pages
pages = []

for url in crawl_list: 
  page = url.split("/") [-1] # extract the persons name at the end of the url 
  print('Looking at file %s' % page)
  # split the url into different parts, and convert the persons name into ascii code
  url_list = list(urllib.parse.urlsplit(url))
  url_list[2] = urllib.parse.quote(url_list[2])
  url_ascii = urllib.parse.urlunsplit(url_list)
  try:
    response = urllib.request.urlopen((url_ascii))
    #Save the page and url for later use
    pages.append(response)
  except urllib.error.URLError as e:
    print(e.reason)

Looking at file Julie_Sweet.html
Looking at file Kumar_Birla.html
Looking at file Shantanu_Narayen.html
Looking at file Garo_H._Armen.html
Looking at file Tom_Enders.html
Looking at file Daniel_Zhang.html
Looking at file Jeff_Bezos.html
Looking at file Lisa_Su.html
Looking at file Stephen_Squeri.html
Looking at file Doug_Parker.html
Looking at file Joseph_R._Swedish.html
Looking at file Tim_Cook.html
Looking at file Lakshmi_Niwas_Mittal.html
Looking at file Randall_L._Stephenson.html
Looking at file Charles_Woodburn.html
Looking at file Tapan_Singhel.html
Looking at file Carlos_Torres_Vila.html
Looking at file Brian_Moynihan.html
Looking at file Jes_Staley.html
Looking at file Warren_Buffett.html
Looking at file Hubert_Joly.html
Looking at file Sunil_Bharti_Mittal.html
Looking at file Stephen_A._Schwarzman.html
Looking at file Andrew_Mackenzie.html
Looking at file Harald_Krüger.html
Looking at file Charles_Scharf.html
Looking at file Dennis_Muilenburg.html
Looking at file Rich_Lesser.h

In [ ]:
pages

Populate a table with the name, we page URL, and birth date of each CEO

In [ ]:
# Use etree.HTML(...) on the HTML content of each page to get a DOM tree that
# can be processed via XPath to extract the bday information. Store the CEO name, 
# webpage, and the birthday (born) in exec_df

# We first check that the HTML content has a table type vcard
# and then extract the bday information. If there is no birthday, the datetime 
# values is Nat (not a type)

exec_df = pd.DataFrame(columns=['name','page','born'])

for page in pages: 

  tree = etree.HTML(page.read().decode("utf-8")) #creatinf a DOM tree of the pages
  url = page.geturl()
  bday = tree.xpath('//table[contains(@class,"vacard")]//tbody/tr/td/span[@class="bday"]/text()')
  if (len(bday)>0):
    exec_df = exec_df.append({'name': url[url.rfind('/')+1:-5], 'page': url, 
                              'born': datetime.datetime.strptime(bday[0], '%Y-%m-%d')}, ignore_index=True)
  else:
    exec_df = exec_df.append({'name': url[url.rfind('/')+1:-5], 'page':url
                             , 'born': None}, ignore_index=True)
    
exec_df

,name,page,born
0,Julie_Sweet,https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autolo...,None
1,Kumar_Birla,https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autolo...,None
2,Shantanu_Narayen,https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autolo...,None
3,Garo_H._Armen,https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autolo...,None
4,Tom_Enders,https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autolo...,None
...,...,...,...
177,Laura_J._Alber,https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autolo...,None
178,Abidali_Neemuchwala,https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autolo...,None
179,Vince_McMahon,https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autolo...,None
180,Greg_Creed,https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autolo...,None


# Cleaning and Manipulating the Tables

In [ ]:
# Let's look at the name and born data in exec_df. We do this via projection. 
# Notice also (e.g row 24) that special characters will cause issues 

exec_df[['name', 'born']]

,name,born
0,Julie_Sweet,None
1,Kumar_Birla,None
2,Shantanu_Narayen,None
3,Garo_H._Armen,None
4,Tom_Enders,None
...,...,...
177,Laura_J._Alber,None
178,Abidali_Neemuchwala,None
179,Vince_McMahon,None
180,Greg_Creed,None


In [ ]:
#Clean the name by removing underscores, and store in a new field called clean_name

exec_df['clean_name'] = exec_df['name'].apply(lambda x: x.replace('_', ' '))

exec_df

,name,page,born,clean_name
0,Julie_Sweet,https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autolo...,None,Julie Sweet
1,Kumar_Birla,https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autolo...,None,Kumar Birla
2,Shantanu_Narayen,https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autolo...,None,Shantanu Narayen
3,Garo_H._Armen,https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autolo...,None,Garo H. Armen
4,Tom_Enders,https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autolo...,None,Tom Enders
...,...,...,...,...
177,Laura_J._Alber,https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autolo...,None,Laura J. Alber
178,Abidali_Neemuchwala,https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autolo...,None,Abidali Neemuchwala
179,Vince_McMahon,https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autolo...,None,Vince McMahon
180,Greg_Creed,https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autolo...,None,Greg Creed


In [ ]:
# We can do the same via SQL. For example, we'll save the dataframe first, and then extract just the name and clean_name using SQL

exec_df.to_sql('temp_exec', conn, if_exists="replace")

pd.read_sql_query('select name, replace(name, "_", " ") as clean_name from temp_exec', conn)

,name,clean_name
0,Julie_Sweet,Julie Sweet
1,Kumar_Birla,Kumar Birla
2,Shantanu_Narayen,Shantanu Narayen
3,Garo_H._Armen,Garo H. Armen
4,Tom_Enders,Tom Enders
...,...,...
177,Laura_J._Alber,Laura J. Alber
178,Abidali_Neemuchwala,Abidali Neemuchwala
179,Vince_McMahon,Vince McMahon
180,Greg_Creed,Greg Creed


In [ ]:
exec_df['clean_name']

0              Julie Sweet
1              Kumar Birla
2         Shantanu Narayen
3            Garo H. Armen
4               Tom Enders
              ...         
177         Laura J. Alber
178    Abidali Neemuchwala
179          Vince McMahon
180             Greg Creed
181        Spencer Rascoff
Name: clean_name, Length: 182, dtype: object

In [ ]:
exec_df['clean_name']=='Kumar Birla'

0      False
1       True
2      False
3      False
4      False
       ...  
177    False
178    False
179    False
180    False
181    False
Name: clean_name, Length: 182, dtype: bool

In [ ]:
exec_df[exec_df['clean_name']=='Kumar Birla']

,name,page,born,clean_name
1,Kumar_Birla,https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autolo...,None,Kumar Birla


In [ ]:
exec_df.to_sql('temp_exec', conn, if_exists='replace')
pd.read_sql_query('select * from temp_exec where clean_name="Kumar Birla"', conn)

,index,name,page,born,clean_name
0,1,Kumar_Birla,https://raw.githubusercontent.com/odpi/OpenDS4All/penn-processing-zgi/assets/data/scripts-autolo...,None,Kumar Birla
